In [ ]:
from model import ModelFactory
from torch import nn
from torch import Tensor
import pickle
import torch

model_factory = ModelFactory(
    coordinates = 100,
    words = 101,
    tokens=50258 + 3,
    number_of_blocks = 10,
    
    number_of_heads = 3,
    bias = False,
    attention = "metric"# "scaled_dot_product", # or "metric"
)

class SentimentModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = model_factory.create_model(kind="encoder")
        del self.model[-1]
        del self.model[-1]
        self.classification_head = nn.Sequential(
            nn.LayerNorm(model_factory.coordinates),
            nn.Linear(model_factory.coordinates, model_factory.coordinates),
            nn.GELU(),
            nn.Linear(model_factory.coordinates, model_factory.coordinates),
            nn.GELU(),
            nn.Linear(model_factory.coordinates // 2, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, sequence_bw: Tensor) -> Tensor:
        sequence_bwc = self.model(sequence_bw)
        sequence_bc = sequence_bwc.mean(dim=1) # average over words
        sequence_bv = self.classification_head(sequence_bc)
        return sequence_bv

with open("stanfordSentimentTreebank.pickle", "rb") as f:
    dataset = pickle.load(f)

sentence_Bw = dataset['train'][0]
sentence_Bw = torch.tensor(sentence_Bw)
print(sentence_Bw)

SentimentModel()(sentence_Bw[0:32])


tensor([[  464,  4631,   318,  ..., 50258, 50258, 50258],
        [  464, 17177,  3481,  ..., 50258, 50258, 50258],
        [   50,  3889, 11139,  ..., 50258, 50258, 50258],
        ...,
        [ 1135,   705,   303,  ..., 50258, 50258, 50258],
        [ 9360,  3296,  6807,  ..., 50258, 50258, 50258],
        [  818,   428,  1339,  ..., 50258, 50258, 50258]])


: 

In [ ]:

print("here")


def yieldtrainingdata():

    sentences, sentiments = dataset['train']
    
    size = len(sentences)
    for i in range(0, size, 32):
        yield torch.tensor(sentences[i:i+32]), torch.tensor(sentiments[i:i+32])


model = SentimentModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

for epoch in range(10):
    for sentence_bw, sentiment_b in yieldtrainingdata():
        optimizer.zero_grad()
        pred_logits_bv = model(sentence_bw)
        loss_train = loss_function(pred_logits_bv, sentiment_b)
        loss_train.backward()
        optimizer.step()
        print(loss_train)
        

here


NameError: name 'SentimentModel' is not defined

In [ ]:
import tiktoken
# special_tokens
gpt2_encoder = tiktoken